In [29]:
import pandas as pd
import matplotlib as plt
import numpy as np

Importing both datasets

In [30]:
hawkeye = pd.read_stata('/Users/lucaslippman/Daniel/MergeDaniel/HKdata_3matches.dta')

challenge = pd.read_stata('/Users/lucaslippman/Daniel/MergeDaniel/Challenges_3matches.dta')


First Thing I coded here was the algorithim to find the nearest line that the ball landed in. This was incorporated with the hawkeye dataset

In [31]:
import math
def find_nearest_line(coord):
    x, y = coord
    distances = {
        "x=-11.8": abs(x + 11.8),
        "x=11.8": abs(x - 11.8),
        "y=4.115": abs(y - 4.115),
        "y=-4.115": abs(y + 4.115),
        "x=6.4": abs(x + 6.4),
        "x=-6.4": abs(x - 6.4),
        "x=0": abs(x),
        "y=0": abs(y)
    }
    closest_line = min(distances, key=distances.get)
    return closest_line

def find_nearest_acronym(coord):
    x, y = coord
    distances = {
        "x=-11.8": abs(x + 11.8),
        "x=11.8": abs(x - 11.8),
        "y=4.115": abs(y - 4.115),
        "y=-4.115": abs(y + 4.115),
        "x=6.4": abs(x + 6.4),
        "x=-6.4": abs(x - 6.4),
        "x=0": abs(x),
        "y=0": abs(y)
    }
    closest_line = min(distances, key=distances.get)
    
    if closest_line in ["y=4.115", "y=-4.115"]:
        acronym = "F"
    elif closest_line in ["x=-11.8", "x=11.8"]:
        acronym = "B"
    elif closest_line in ["x=-6.4", "x=6.4"]:
        acronym = "S"
    elif closest_line == "x=0":
        acronym = "N"
    elif closest_line == "y=0" and -11.8 < x < 11.8:
        acronym = "C"
    else:
        acronym = "B" if x < -11.8 or x > 11.8 else "C"
        
    return acronym

def find_label(coord):
    x, y = coord
    closest_line = find_nearest_line(coord)
    if closest_line == "y=0" and -11.8 < x < 11.8:
        acronym = "C"
    else:
        acronym = find_nearest_acronym(coord)
    if x == 0:
        side = ""
    elif x < 0:
        side = "R"
    else:
        side = "L"
    return side + acronym


hawkeye['label'] = hawkeye.apply(lambda row: find_label((row['x1'], row['y1'])), axis=1)


Making some columns for hawkeye, like set which adds up the current scores of sets +1. (so if set is 0 0 it will be set 1 ). Also, add up games within a set. For example, 5+4 = 9, so that its easy to match with challenge dataset

In [32]:
hawkeye["set"] = hawkeye["scoresetserver"] + hawkeye["scoresetreceiver"]
hawkeye["sum_games"] = hawkeye["scoregameserver"] + hawkeye["scoregamereceiver"]
hawkeye["dist_abs"] = abs(hawkeye["dist"])


Making some more new columns. "Call" is a column that takes hawk_in and converts it to in and out. I did this for my viewing purposes. Also, i divided the distance by 1000 so it would be in metres and not mm. 

In [33]:
challenge["call"] = challenge["hawk_in"].apply(lambda x: "in" if x == 1 else "out")
challenge["distance"] = challenge["distance_mm"] / 1000


This next code added sum_games to the challenge dataset. I did this by checking what the first set without NAN's was and then adding those values up. 

In [34]:
def add_sets(row):
    if pd.notna(row["set3_chal"]) and pd.notna(row["set3_opp"]):
        return row["set3_chal"] + row["set3_opp"]
    elif pd.notna(row["set2_chal"]) and pd.notna(row["set2_opp"]):
        return row["set2_chal"] + row["set2_opp"]
    else:
        return row["set1_chal"] + row["set1_opp"]
    
    
challenge["sum_games"] = challenge.apply(add_sets, axis=1)
challenge["out"] = challenge["call"].replace({'out': 1, 'in': 0})


Exported the hawkeye and challenge datasets, now reading them back into the code

In [35]:
challenge = pd.read_csv("/Users/lucaslippman/Daniel/challenge.csv")
hawkeye = pd.read_csv("/Users/lucaslippman/Daniel/hawkeyey.csv")
hawkeye['set']= hawkeye['set'] +1

This is code that converts tennis points into values. I make a key assumption here because our challenge dataset doesn't give us the values of advantage points. In order to match these points then, we must assume that all values above 3 are "50"

In [36]:
def tennis_points(value):
    value = int(value)
    if value == 0:
        return 0
    elif value == 1:
        return 15
    elif value == 2:
        return 30
    elif value == 3:
        return 40
    else:
        return 50

# Split the 'score' column into two columns
hawkeye[['server_score', 'receiver_score']] = hawkeye['score'].str.split(' ', expand=True)

# Convert the tennis points to their actual values
hawkeye[['server_score', 'receiver_score']] = hawkeye[['server_score', 'receiver_score']].applymap(tennis_points)


Make a new column called "comb_game_x" which adds up the points within a game. Example: 40 30 = 70. This will make easy matching with challenge dataset. 

In [37]:
hawkeye['comb_game_x'] = hawkeye['server_score'] + hawkeye['receiver_score']


This code limits the values of the comb_game_x  to 90 since we have no way of matching these values with challenge dataset if not. Since challenge data set does ADV points like NAN 50, we make that value 90 as well. Also, if there is a scoreline that is NAN NAN, then that is assumed to be 0 0 according to our research.

In [38]:
hawkeye['comb_game_x'] = hawkeye['comb_game_x'].apply(lambda x: min(x, 90))

# Add the 'comb_game_y' column based on the conditions
challenge['comb_game_y'] = challenge.apply(lambda row: 90 if row['game_chal'] == 50 or row['game_opp'] == 50 else row['game_chal'] + row['game_opp'], axis=1)

hawkeye['comb_game_x'] = hawkeye['comb_game_x'].astype(float)
challenge['comb_game_y'] = challenge['comb_game_y'].astype(float)
challenge['comb_game_y'] = challenge['comb_game_y'].fillna(0)



Now, we will merge the datasets together to find rows that match. We do this by matching it through the set, the games, and the torunament name first. This reduces the dataframe down to 1332 rows. Also, adds some rows. number_hawkeye shows which hawkeye row has been merged. number_challenge shows the number row of the challenge dataset

In [39]:
def merge_condition(row):
    return (
        row["set_x"] == row["set_y"]
        and row["sum_games_x"] == row["sum_games_y"]
        and row['tournament_x'] == row['tournament_y']

    )

    

# Perform a cross join to combine all possible pairs of rows
hawkeye["key"] = 1
challenge["key"] = 1
merged_df = pd.merge(hawkeye, challenge, on="key").drop("key", axis=1)

# Filter the rows based on the merge_condition
merged_df["merge_condition"] = merged_df.apply(merge_condition, axis=1)
merged_df = merged_df[merged_df["merge_condition"]]

# Drop the duplicate columns and the merge_condition column
merged_df = merged_df.drop(
    columns=["set_y", "sum_games_y", "out_y", "merge_condition"]
)

# Rename the 'number' column from the 'challenge' DataFrame to avoid confusion
merged_df = merged_df.rename(columns={"number": "number_hawkeye", "Unnamed: 0": "number_challenge"})
cols_to_move = ['number_hawkeye', 'number_challenge','dist_abs','score','game_chal','game_opp', 'hawk_in','challenge_won','comb_game_x','comb_game_y']
merged_df = merged_df[[col for col in merged_df.columns if col not in cols_to_move] + cols_to_move]
merged_df = merged_df.sort_values(by="number_challenge")


In [18]:
merged_df

,server,receiver,scorer,impact,x1,y1,foldername_x,tournament_x,iiid,gender,...,number_hawkeye,number_challenge,dist_abs,score,game_chal,game_opp,hawk_in,challenge_won,comb_game_x,comb_game_y
0,LOPEZ,DAVYDENKO,-1,0.0,-6.034672,-0.344506,SemiLopezDavydenko,DubaiMens08,570954,male,...,765,0,0.393968,0 0,15.0,40.0,0,0,0.0,55.0
1,LOPEZ,DAVYDENKO,-1,3.0,10.177990,2.713394,SemiLopezDavydenko,DubaiMens08,570955,male,...,766,0,1.430063,0 0,15.0,40.0,0,0,0.0,55.0
2,LOPEZ,DAVYDENKO,-1,6.0,-10.100620,-2.540116,SemiLopezDavydenko,DubaiMens08,570956,male,...,767,0,1.604649,0 0,15.0,40.0,0,0,0.0,55.0
3,LOPEZ,DAVYDENKO,-1,9.0,4.939360,-2.517123,SemiLopezDavydenko,DubaiMens08,570957,male,...,768,0,1.620632,0 0,15.0,40.0,0,0,0.0,55.0
4,LOPEZ,DAVYDENKO,-1,12.0,-9.783928,4.180157,SemiLopezDavydenko,DubaiMens08,570958,male,...,769,0,0.029688,0 0,15.0,40.0,0,0,0.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,ANCIC,MURRAY,-1,30.0,7.929305,-2.494188,FinalMurayAncic,Marseille08,1483712,male,...,2909,33,1.638126,2 3,0.0,0.0,1,1,70.0,0.0
1328,ANCIC,MURRAY,-1,NaN,NaN,NaN,FinalMurayAncic,Marseille08,1483713,male,...,2910,33,NaN,2 3,0.0,0.0,1,1,70.0,0.0
1329,ANCIC,MURRAY,-1,34.0,5.183188,-0.405978,FinalMurayAncic,Marseille08,1483714,male,...,2911,33,3.721728,2 3,0.0,0.0,1,1,70.0,0.0
1330,ANCIC,MURRAY,-1,NaN,NaN,NaN,FinalMurayAncic,Marseille08,1483715,male,...,2912,33,NaN,2 3,0.0,0.0,1,1,70.0,0.0


Here, for every number_challenge possible, I am organizing the rows of hawkeye. I am doing this so the last hawkeye row of a point will be at the end so that we can determine what is the last point in a score and the first point. 

In [40]:
merged_df.sort_values(['number_challenge', 'number_hawkeye'], inplace=True)

# reset the index
merged_df.reset_index(drop=True, inplace=True)

Here, I am adding conditions that reduce the dataset down to serves and final points of scores. I use the idinpoint ==1 to find serves, and for final ball in a score, i identify when a new point starts and shift that marker up 1 row. 

In [42]:
merged_df['comb_game_x_change'] = ((merged_df['comb_game_x'].diff() != 0))
merged_df['comb_game_y_change'] = ((merged_df['idinpoint'] == 1))

merged_df['comb_game_x_change'] = merged_df['comb_game_x_change'].shift(-1).fillna(0).astype(int)
merged_df.dropna(subset=['x1', 'y1'], inplace=True)


This creates the dataframe with the restrictions of serves and final points of . Here is the new dataframe below. We go from 1332 to 473

In [44]:
filtered_df = merged_df[(merged_df['comb_game_x_change'] == 1) | (merged_df['comb_game_y_change'] == True)]
filtered_df

,server,receiver,scorer,impact,x1,y1,foldername_x,tournament_x,iiid,gender,...,dist_abs,score,game_chal,game_opp,hawk_in,challenge_won,comb_game_x,comb_game_y,comb_game_x_change,comb_game_y_change
0,LOPEZ,DAVYDENKO,-1,0.0,-6.034672,-0.344506,SemiLopezDavydenko,DubaiMens08,570954,male,...,0.393968,0 0,15.0,40.0,0,0,0.0,55.0,0,True
5,LOPEZ,DAVYDENKO,-1,15.0,13.387200,2.918488,SemiLopezDavydenko,DubaiMens08,570959,male,...,1.406800,0 0,15.0,40.0,0,0,0.0,55.0,1,False
6,LOPEZ,DAVYDENKO,1,0.0,-6.712499,1.182852,SemiLopezDavydenko,DubaiMens08,570960,male,...,0.173002,0 1,15.0,40.0,0,0,15.0,55.0,0,True
8,LOPEZ,DAVYDENKO,-1,0.0,-5.699979,2.568540,SemiLopezDavydenko,DubaiMens08,570962,male,...,0.825438,0 1,15.0,40.0,0,0,15.0,55.0,0,True
10,LOPEZ,DAVYDENKO,-1,6.0,-8.020497,-4.761695,SemiLopezDavydenko,DubaiMens08,570964,male,...,0.604794,0 1,15.0,40.0,0,0,15.0,55.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,ANCIC,MURRAY,0,0.0,6.387270,4.442170,FinalMurayAncic,Marseille08,1483692,male,...,0.145400,2 2,0.0,0.0,1,1,60.0,0.0,0,True
1308,ANCIC,MURRAY,-1,0.0,5.940933,3.678801,FinalMurayAncic,Marseille08,1483693,male,...,0.591172,2 2,0.0,0.0,1,1,60.0,0.0,0,True
1315,ANCIC,MURRAY,-1,21.0,12.734770,-1.847751,FinalMurayAncic,Marseille08,1483700,male,...,0.780310,2 2,0.0,0.0,1,1,60.0,0.0,1,False
1316,ANCIC,MURRAY,0,0.0,6.009051,-4.317004,FinalMurayAncic,Marseille08,1483701,male,...,0.525740,2 3,0.0,0.0,1,1,70.0,0.0,0,True


Now we have multiple possibities to match these points, either its a serve or its not a serve!. What I did here is create an algorithim that matches serves in both datasets and final points in both datasets. I do this by converting the idinpoint to a binary variable: if idinpoint ==1 thats a serve. If it doesn't, thats the final point. And we have that variable in the challenge dataset. So, all we have to do is match those two. Also, I made sure to remove the potential replayed points from this classification as if a point is replayed we don't know if it will be in the dataset.

In [46]:
def filter_rows(group):
    # Convert 'idinpoint' to binary variable: 1 if it is a serve, 0 if it is not a serve
    group['idinpoint'] = group['idinpoint'].apply(lambda x: 1 if x == 1 else 0)
    
    # Identify rows where 'challenge_won' and 'comb_game_x' are both 1
    special_rows = (group['challenge_won'] == 1) & (group['comb_game_x'] == 1)
    
    # Check if there are any rows where 'idinpoint' matches 'serve', or it's a special row
    matches = special_rows | (group['idinpoint'] == group['serve'])
    
    # If there are any matches or 'idinpoint' and 'serve' are both not 1, return those rows
    # If there are no matches, return all rows
    return group[matches | ((group['idinpoint'] != 1) & (group['serve'] != 1))] if matches.any() else group

# Group the dataframe by 'number_challenges_2' and 'score' and apply the 'filter_rows' function to each group
df = filtered_df.groupby(['number_challenge', 'score']).apply(filter_rows).reset_index(drop=True)

Now, we will merge our dataset down to the 1 to 1. I have multiple steps to do this. First, my algorithim finds if there are any points that match in both datasets (example 15 15 and 15 15) and then reduces the rows down to that. After that, the algorithim finds the closest distance of each point. If there aren't any matching points in both datasets, then the algorithim finds the closest distance to match. 

In [47]:
def select_desired_row(group):
    # Select rows where 'comb_game_x' and 'comb_game_y' are equal
    same_comb_game = group[group['comb_game_x'] == group['comb_game_y']]
    
    # If there are suitable 'comb_game_x' and 'comb_game_y' values
    if not same_comb_game.empty:
        # Find the index of the row with the minimum difference between 'dist_abs' and 'distance' for suitable rows
        min_diff_index = (same_comb_game['dist_abs'] - same_comb_game['distance']).abs().idxmin()
        # Return the row with the minimum difference for suitable rows
        return same_comb_game.loc[min_diff_index]
    else:
        # If either 'comb_game_x' or 'comb_game_y' has missing values (NaN) in the group, or there are no suitable rows
        if group['comb_game_x'].isna().any() or group['comb_game_y'].isna().any() or same_comb_game.empty:
            # Find the index of the row with the minimum difference between 'dist_abs' and 'distance'
            min_diff_index = (group['dist_abs'] - group['distance']).abs().idxmin()
            # Return the row with the minimum difference
            return group.loc[min_diff_index]

# Group the dataframe by 'number_challenges_2' and apply the 'select_desired_row' function to each group


result = filtered_df.groupby('number_challenge').apply(select_desired_row).reset_index(drop=True)

TWO POINTS WERE NOT ABLE TO BE MATCHED SO THE ALGORITHIM FINDS NEAREST DISTANCE. THose points are row 30 and row 17

In [48]:
result

,server,receiver,scorer,impact,x1,y1,foldername_x,tournament_x,iiid,gender,...,dist_abs,score,game_chal,game_opp,hawk_in,challenge_won,comb_game_x,comb_game_y,comb_game_x_change,comb_game_y_change
0,LOPEZ,DAVYDENKO,-1,0.0,-6.651841,-1.147098,SemiLopezDavydenko,DubaiMens08,570970,male,...,0.123161,1 3,15.0,40.0,0,0,55.0,55.0,0,True
1,DAVYDENKO,LOPEZ,1,0.0,-6.414097,2.683151,SemiLopezDavydenko,DubaiMens08,570793,male,...,0.124471,1 0,0.0,15.0,1,0,15.0,15.0,0,True
2,LOPEZ,DAVYDENKO,0,0.0,6.629273,3.591878,SemiLopezDavydenko,DubaiMens08,570225,male,...,0.092159,0 2,0.0,30.0,0,0,30.0,30.0,0,True
3,LOPEZ,DAVYDENKO,-1,0.0,6.443140,0.441272,SemiLopezDavydenko,DubaiMens08,570919,male,...,0.097317,2 0,15.0,15.0,0,1,30.0,30.0,0,True
4,DAVYDENKO,LOPEZ,-1,18.0,10.686390,-4.157564,SemiLopezDavydenko,DubaiMens08,570420,male,...,0.009434,0 1,0.0,15.0,0,0,15.0,15.0,1,False
5,LOPEZ,DAVYDENKO,-1,0.0,-6.034672,-0.344506,SemiLopezDavydenko,DubaiMens08,570954,male,...,0.393968,0 0,NaN,NaN,0,0,0.0,0.0,0,True
6,DAVYDENKO,LOPEZ,0,0.0,-6.606727,3.722899,SemiLopezDavydenko,DubaiMens08,570531,male,...,0.074273,2 3,40.0,30.0,0,1,70.0,70.0,0,True
7,DAVYDENKO,LOPEZ,-1,0.0,-5.317419,0.213988,SemiLopezDavydenko,DubaiMens08,570365,male,...,0.262180,0 1,15.0,0.0,0,1,15.0,15.0,0,True
8,LOPEZ,DAVYDENKO,0,0.0,6.607439,4.781987,SemiLopezDavydenko,DubaiMens08,571193,male,...,0.616384,2 2,30.0,30.0,0,0,60.0,60.0,0,True
9,DAVYDENKO,LOPEZ,-1,0.0,-5.497965,0.206658,SemiLopezDavydenko,DubaiMens08,570493,male,...,0.254939,0 1,15.0,0.0,1,0,15.0,15.0,0,True
